**Football is widely considered the most popular sport in the world.**

 With the sports betting industry worth $5.4 billion in the UK alone, accurately forecasting match outcomes can be incredibly lucrative endeavour. \
 Although beating the odds consistently is a notoriously difficult task, we will in this notebook attempt to understand "*The Beautiful Game*" and explore the problem of consistently predicting football match outcomes. \

 We assume, in undertaking this task, that a team's past performance is *somewhat* correlated to its future performance. 

 To start, we will download some basic historical match data freely provided by football-data.co.uk. We will initially download data for the last 5 years in the English Premier League, and then explore the effect on model accuracy when using data from a larger/smaller set of years and/or a larger set of leagues. We will use the Pandas library to conveniently read the CSVs hosted on the side directly into in-memory DataFrames and the datetime library to easily get the current year in order to fetch the most recent data.


In [162]:
import pandas as pd
from datetime import datetime

import requests

def get_basic_match_data(season_start_year: str = "2017", season_end_year: str = str(datetime.now().year), countries: dict[str, list[str|int]] = {"England": [1, 2], "Spain": [1], "Germany": [1], "Italy": [1], "France": [1]}, get_odds: bool = False):
    """Download match data from a provider (currently only https://football-data.co.uk/)\n
    If this function returns an URLError , rerun it using a VPN as some network providers block the site.\n
    The definitions of each column can be found at https://football-data.co.uk/notes.txt \n

    Parameters
    ----------    
    `data_start_year` The first year to download data for. An input of 2017 would have a first season of 1718. Defaults to 2017.\n 
    `data_end_year` The last year to download data for. An input of 23 would have a final season of 2223. Defaults to the current year.\n
    `countries` A dictionary of all the countries you would like to include, and the divisions from each country. The keys are the country names and the values are a list of the divisions desired from each country: 1 represents the top flight, 2 the second highest league etc. 
    Default value will return the top flights of 'The Big 5' i.e England, Spain, Italy, Germany and France.\n 
    `get_odds` Bool. If True, include odds data in the returned DataFrame. Defaults to True.
    Returns
    ---------- 
    A DataFrame containing every match played in all requested divisions from the given start year to the given end year.

    """
    # Please note that this method is dependent on the urls it is served by. Should a data provider change the location they host data downloaded by this method, it will not work until updated accordingly. 
    # Although scraping providers is a possible alternative, changes to their site structure would then cause the method to raise an error.
    # Optionally caching downloaded data is another possible solution, although not necessary for this demonstration. 
    
    #Each country has different division codes, so we create a buffer dict 'new_countries' to hold the new, standardised country-division pairs while iterating over the inital dict.
    new_countries = {}
    for country in countries:
        if country == "Germany":
            new_countries["D"] = countries[country]
        elif country == "Scotland":
            new_countries["SC"] = [div-1 for div in countries[country]]
        elif country == "Spain":
            new_countries["SP"] = countries[country]
        elif country == "England":
            new_countries[country[0].upper()] = [div-1 for div in countries[country]]
            
        else:
            new_countries[country[0].upper()] = countries[country]

    countries = new_countries
    del new_countries
    #print([f"https://www.football-data.co.uk/mmz4281/{season-1}{season}/{country}{division}.csv" for season in range(int(season_start_year[2:]), int(season_end_year[2:])) for country in countries for division in countries[country]])
    
    conditional_args = {}
    if not get_odds: #If we don't want odds data, we only download the following columns (since most other columns are either inconsistently available or odds-related) 
        conditional_args["usecols"] = ["Date", "Div", 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HY", "AY", "HR", "AR"]

    #The following line returns a DataFrame concatenated from a list comprehension of smaller DataFrames each representing a season in a particular division, e.g 1717 in E0(the English Premier League) 
    return pd.concat([pd.read_csv(f"https://www.football-data.co.uk/mmz4281/{season}{season+1}/{country}{division}.csv", index_col=["Div", "Date"], parse_dates=True, dayfirst=True, **conditional_args) for season in range(int(season_start_year[2:]), int(season_end_year[2:])) for country in countries for division in countries[country]], \
                     keys= [int(f"{season}{season + 1}") for season in range(int(season_start_year[2:]), int(season_end_year[2:])) for country in countries for div in countries[country]], names=["Season"])

matchDf = get_basic_match_data()


HomeTeam      AwayTeam  FTHG  FTAG FTR  HTHG  \
Season Div Date                                                             
1718   E0  2017-08-11         Arsenal     Leicester     4     3   H   2.0   
           2017-08-12        Brighton      Man City     0     2   A   0.0   
           2017-08-12         Chelsea       Burnley     2     3   A   0.0   
           2017-08-12  Crystal Palace  Huddersfield     0     3   A   0.0   
           2017-08-12         Everton         Stoke     1     0   H   1.0   
...                               ...           ...   ...   ...  ..   ...   
2223   F1  2023-02-01            Lens          Nice     0     1   A   0.0   
           2023-02-01            Lyon         Brest     0     0   D   0.0   
           2023-02-01          Monaco       Auxerre     3     2   H   2.0   
           2023-02-01     Montpellier      Paris SG     1     3   A   0.0   
           2023-02-01          Rennes    Strasbourg     3     0   H   2.0   

                       HTAG HTR    HS    AS   HST  AST    HF    AF    HC  \
Season Div Date                                                            
1718   E0  2017-08-11   2.0   D  27.0   6.0  10.0  3.0   9.0  12.0   9.0   
           2017-08-12   0.0   D   6.0  14.0   2.0  4.0   6.0   9.0   3.0   
           2017-08-12   3.0   A  19.0  10.0   6.0  5.0  16.0  11.0   8.0   
           2017-08-12   2.0   A  14.0   8.0   4.0  6.0   7.0  19.0  12.0   
           2017-08-12   0.0   H   9.0   9.0   4.0  1.0  13.0  10.0   6.0   
...                     ...  ..   ...   ...   ...  ...   ...   ...   ...   
2223   F1  2023-02-01   0.0   D   9.0  14.0   4.0  6.0  12.0   8.0   5.0   
           2023-02-01   0.0   D  28.0   8.0   8.0  1.0  10.0  13.0  15.0   
           2023-02-01   0.0   H  16.0  11.0   6.0  3.0  11.0   7.0   5.0   
           2023-02-01   0.0   D   6.0  19.0   3.0  8.0   9.0  12.0   1.0   
           2023-02-01   0.0   H   6.0   3.0   3.0  0.0  12.0  13.0   4.0   

                         AC   HY   AY   HR   AR  
Season Div Date                                  
1718   E0  2017-08-11   4.0  0.0  1.0  0.0  0.0  
           2017-08-12  10.0  0.0  2.0  0.0  0.0  
           2017-08-12   5.0  3.0  3.0  2.0  0.0  
           2017-08-12   9.0  1.0  3.0  0.0  0.0  
           2017-08-12   7.0  1.0  1.0  0.0  0.0  
...                     ...  ...  ...  ...  ...  
2223   F1  2023-02-01   3.0  0.0  2.0  0.0  0.0  
           2023-02-01   2.0  0.0  0.0  0.0  0.0  
           2023-02-01   4.0  0.0  0.0  0.0  0.0  
           2023-02-01   3.0  1.0  1.0  0.0  0.0  
           2023-02-01   0.0  0.0  1.0  0.0  0.0  

[13087 rows x 20 columns]

Now that we have tabular data consisting of all the matches that occurred in the seasons bounded by our desired start and end years, we should reshape it into a format that can more naturally produce features with predictive power. \
As stated in the previous text cell, the assumption underlying this notebook (and much of the general field of sports analytics) is that a team's (generalised to an entity, a term that can also encompass players, managers etc in the wider field) past performance bears some variable relationship to its future performance. This is anecdotally true - anyone slightly knowledgeable about football will tell you that in a match between Manchester City and Leyton Orient, for instance, City are far more likely to win (justifying this prediction with a multitude of reasons which mostly relate to the past performance of both teams). 

According to the argument above it is most reasonable to reshape our data, which is currently arranged into bins of seasons and league division code, into bins of teams and seasons. For any of our table's columns to actually encompass a feature of a team's past performance, they must be aggregated as each column presently only represents a team's performance in a single match and not, as we would like, a team's performance over a set, longer period of time (i.e a season or last 5 matches)\
Our data is currently organised by match, detailing every match's home and away team and various statistics describing aspects of their performance in that match. We want our data to be organised by team, so we must combine and aggregate each team's home and away stats - which should be kept separate because a team's performance (as we will find later in this notebook) can be influenced by the stadium it is playing in.


We achieve this in the following cells.


In [228]:
#Returns a DataFrame containing the cumulative sum of each statistic(Home and Away) over all their seasons in the leagues we downloaded data on by concatenating a HomeTeam indexed DataFrame and an AwayTeam indexed DataFrame. on their columns
df = pd.concat([matchDf.drop(columns=[team, "FTR", "HTR"] + [f"{team[0]}{x}" for x in ["S", "ST", "F", "C", "Y", "R"]] + [f"FT{team[0]}G", f"HT{team[0]}G"]).reset_index().set_index(list(set(("HomeTeam", "AwayTeam")) - set([team])) + ["Season", "Div", "Date"]).  #We create 2 DataFrames, one indexed by each team's home games and the other by each team's away games. We drop away columns for the home-indexed Df and vice versa. We also drop the result columns(FTR and HTR) as they are categorical.
                rename_axis(index= ["Team", "Season", "Div", "Date"]).groupby("Team").transform(lambda x:                  #We group the DataFrame by team, so that in the next line we can separately aggregate each team's performance 
        x.expanding().mean().shift().fillna(0)).sort_values(["Div", "Season", "Date"]) for team in ("HomeTeam", "AwayTeam")], axis = 1).sort_index()   #We aggregate each team's statistics by taking the cumulative sum over every match they have played, and shift each team's statistics forward a match to avoid data leakage as otherwise models would see the impact of the game they are trying to predict the outcome of on each team's statistics.

"""matchDf.drop(columns=["AwayTeam", "FTR", "HTR"]).groupby(["HomeTeam"]).transform(lambda team: team.expanding().mean().shift().fillna(0)).sort_values(["Div", "Season", "HomeTeam", "Date"])
matchDf.drop(columns=["HomeTeam", "FTR", "HTR"]).groupby(["AwayTeam"]).transform(lambda team: team.expanding().mean().shift().fillna(0)).sort_values(["Div", "Season", "HomeTeam", "Date"])"""
df

FTAG      HTAG         AS       AST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05  0.000000  0.000000   0.000000  0.000000   
                   2022-08-14       NaN       NaN        NaN       NaN   
                   2022-08-21  1.000000  1.000000   8.000000  4.000000   
                   2022-08-26       NaN       NaN        NaN       NaN   
                   2022-08-31  1.000000  0.500000   9.000000  3.500000   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17  0.800000  0.300000  10.150000  2.700000   
                   2021-04-21       NaN       NaN        NaN       NaN   
                   2021-04-24  0.857143  0.285714  10.380952  2.809524   
                   2021-05-01       NaN       NaN        NaN       NaN   
                   2021-05-08  0.863636  0.318182  10.590909  2.772727   

                                      AF        AC       AY        AR  \
Team    Season Div Date                                                 
Ajaccio 2223   F1  2022-08-05   0.000000  0.000000  0.00000  0.000000   
                   2022-08-14        NaN       NaN      NaN       NaN   
                   2022-08-21  18.000000  3.000000  6.00000  1.000000   
                   2022-08-26        NaN       NaN      NaN       NaN   
                   2022-08-31  14.500000  5.500000  4.00000  0.500000   
...                                  ...       ...      ...       ...   
Wycombe 2021   E1  2021-04-17  13.000000  3.700000  1.45000  0.100000   
                   2021-04-21        NaN       NaN      NaN       NaN   
                   2021-04-24  13.047619  3.761905  1.52381  0.095238   
                   2021-05-01        NaN       NaN      NaN       NaN   
                   2021-05-08  13.136364  3.863636  1.50000  0.090909   

                                   FTHG      HTHG         HS       HST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05       NaN       NaN        NaN       NaN   
                   2022-08-14  0.000000  0.000000   0.000000  0.000000   
                   2022-08-21       NaN       NaN        NaN       NaN   
                   2022-08-26  0.000000  0.000000   9.000000  2.000000   
                   2022-08-31       NaN       NaN        NaN       NaN   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17       NaN       NaN        NaN       NaN   
                   2021-04-21  0.666667  0.285714  10.238095  3.190476   
                   2021-04-24       NaN       NaN        NaN       NaN   
                   2021-05-01  0.727273  0.272727  10.772727  3.318182   
                   2021-05-08       NaN       NaN        NaN       NaN   

                                      HF        HC        HY        HR  
Team    Season Div Date                                                 
Ajaccio 2223   F1  2022-08-05        NaN       NaN       NaN       NaN  
                   2022-08-14   0.000000  0.000000  0.000000  0.000000  
                   2022-08-21        NaN       NaN       NaN       NaN  
                   2022-08-26  16.000000  1.000000  5.000000  0.000000  
                   2022-08-31        NaN       NaN       NaN       NaN  
...                                  ...       ...       ...       ...  
Wycombe 2021   E1  2021-04-17        NaN       NaN       NaN       NaN  
                   2021-04-21  14.142857  4.666667  1.571429  0.095238  
                   2021-04-24        NaN       NaN       NaN       NaN  
                   2021-05-01  14.045455  4.681818  1.545455  0.090909  
                   2021-05-08        NaN       NaN       NaN       NaN  

[26174 rows x 16 columns]

Our new DataFrame is arranged in a way far more useful to any predictive models: 
1. It is grouped by team, so data on each team is kept separate. 
2. Each row contains aggregate data on the performance of each team, and keeps the Home and Away performance statistics of each team separately. 
3. Column names for Home and Away statistics follow an intuitive pattern, and so can be aggregated easily.

However, it is also incomplete in a few vital ways:
1. We dropped columns containing categorical data to simplify reshaping the data earlier. The columns dropped were the team, FTR and HTR columns. The FTR column describes the outcome of the match, which we are trying to predict, so we will need to reincorporate it into our new DataFrame.
2. We don't have data on the opposition of a team on a given game. This is necessary as for our model's predictions to be informed, it will need to compare both teams.
3. There is no column explicitly describing whether a team is Home or Away; however since in home games away stats take on NaN values and contrariwise, this can very easily be computed. 

To address these shortcomings, we will take the following steps:
1. Create a column "Home", populated with boolean values detailing whether a team is playing Home or Away.
2. With respect to the lack of data in the opposition, there are 2 viable solutions. We can either "forward fill" NaN values in the current DataFrame to describe each team's Home and Away stats, and then concatenate this DataFrame with another DataFrane structured exactly the same but indexed oppositely in regard of teams (i.e indexed by the opposition of each team for each match) resulting in a new DataFrame with 32 (double the current number of) columns (assuming no other columns have been added or removed) encompassing the Home and Away statistics of both teams. \
The approach we will take is to fill the NaN values in the current DataFrame with data about the opposing side without any forward filling, keeping our DataFrame the same size. The justification for this decision is that larger data tends to be more difficult to work with and is more prone to overfitting. However, this is not absolute and we may later also test our model using the alternative method.
3. We can simply copy the FTR column to the new DataFrame.

In [229]:
df["Home"] = df["FTHG"].notna() #Sets the Home column by deciding a team is home if it has a Full Time Home Goals value (even if that value is 0)
df

FTAG      HTAG         AS       AST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05  0.000000  0.000000   0.000000  0.000000   
                   2022-08-14       NaN       NaN        NaN       NaN   
                   2022-08-21  1.000000  1.000000   8.000000  4.000000   
                   2022-08-26       NaN       NaN        NaN       NaN   
                   2022-08-31  1.000000  0.500000   9.000000  3.500000   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17  0.800000  0.300000  10.150000  2.700000   
                   2021-04-21       NaN       NaN        NaN       NaN   
                   2021-04-24  0.857143  0.285714  10.380952  2.809524   
                   2021-05-01       NaN       NaN        NaN       NaN   
                   2021-05-08  0.863636  0.318182  10.590909  2.772727   

                                      AF        AC       AY        AR  \
Team    Season Div Date                                                 
Ajaccio 2223   F1  2022-08-05   0.000000  0.000000  0.00000  0.000000   
                   2022-08-14        NaN       NaN      NaN       NaN   
                   2022-08-21  18.000000  3.000000  6.00000  1.000000   
                   2022-08-26        NaN       NaN      NaN       NaN   
                   2022-08-31  14.500000  5.500000  4.00000  0.500000   
...                                  ...       ...      ...       ...   
Wycombe 2021   E1  2021-04-17  13.000000  3.700000  1.45000  0.100000   
                   2021-04-21        NaN       NaN      NaN       NaN   
                   2021-04-24  13.047619  3.761905  1.52381  0.095238   
                   2021-05-01        NaN       NaN      NaN       NaN   
                   2021-05-08  13.136364  3.863636  1.50000  0.090909   

                                   FTHG      HTHG         HS       HST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05       NaN       NaN        NaN       NaN   
                   2022-08-14  0.000000  0.000000   0.000000  0.000000   
                   2022-08-21       NaN       NaN        NaN       NaN   
                   2022-08-26  0.000000  0.000000   9.000000  2.000000   
                   2022-08-31       NaN       NaN        NaN       NaN   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17       NaN       NaN        NaN       NaN   
                   2021-04-21  0.666667  0.285714  10.238095  3.190476   
                   2021-04-24       NaN       NaN        NaN       NaN   
                   2021-05-01  0.727273  0.272727  10.772727  3.318182   
                   2021-05-08       NaN       NaN        NaN       NaN   

                                      HF        HC        HY        HR   Home  
Team    Season Div Date                                                        
Ajaccio 2223   F1  2022-08-05        NaN       NaN       NaN       NaN  False  
                   2022-08-14   0.000000  0.000000  0.000000  0.000000   True  
                   2022-08-21        NaN       NaN       NaN       NaN  False  
                   2022-08-26  16.000000  1.000000  5.000000  0.000000   True  
                   2022-08-31        NaN       NaN       NaN       NaN  False  
...                                  ...       ...       ...       ...    ...  
Wycombe 2021   E1  2021-04-17        NaN       NaN       NaN       NaN  False  
                   2021-04-21  14.142857  4.666667  1.571429  0.095238   True  
                   2021-04-24        NaN       NaN       NaN       NaN  False  
                   2021-05-01  14.045455  4.681818  1.545455  0.090909   True  
                   2021-05-08        NaN       NaN       NaN       NaN  False  

[26174 rows x 17 columns]

In [230]:
oppositionsDf = pd.concat([matchDf.set_index([team, matchDf.index]).sort_index().rename_axis(index= ["Team", "Season", "Div", "Date"]) for team in ("HomeTeam", "AwayTeam")])   
oppositionsDf #Produces a DataFrame with the opposition in either the "HomeTeam" or "AwayTeam" columns, depending on whether the opposition is Home or Away. The remaining column takes on a NaN value for that row. 

AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR    HS  \
Team    Season Div Date                                                         
Ajaccio 2223   F1  2022-08-14      Lens     0     0   D   0.0   0.0   D   9.0   
                   2022-08-26     Lille     1     3   A   0.0   2.0   A  13.0   
                   2022-09-04   Lorient     0     1   A   0.0   0.0   D  16.0   
                   2022-09-11      Nice     0     1   A   0.0   0.0   D   9.0   
                   2022-10-02  Clermont     1     3   A   0.0   1.0   A  12.0   
...                                 ...   ...   ...  ..   ...   ...  ..   ...   
Wycombe 2021   E1  2021-03-20       NaN     0     0   D   0.0   0.0   D   9.0   
                   2021-04-05       NaN     0     3   A   0.0   2.0   A  19.0   
                   2021-04-17       NaN     2     2   D   0.0   0.0   D  14.0   
                   2021-04-24       NaN     2     1   H   1.0   1.0   D  10.0   
                   2021-05-08       NaN     0     3   A   0.0   2.0   A   5.0   

                                 AS  HST  AST    HF    AF   HC   AC   HY   AY  \
Team    Season Div Date                                                         
Ajaccio 2223   F1  2022-08-14  12.0  2.0  3.0  16.0  18.0  1.0  3.0  5.0  2.0   
                   2022-08-26   7.0  2.0  4.0  13.0  13.0  6.0  0.0  3.0  2.0   
                   2022-09-04   7.0  3.0  4.0  16.0  12.0  3.0  3.0  4.0  1.0   
                   2022-09-11  12.0  3.0  3.0   9.0  11.0  2.0  3.0  1.0  5.0   
                   2022-10-02   7.0  3.0  3.0  13.0  10.0  4.0  4.0  3.0  3.0   
...                             ...  ...  ...   ...   ...  ...  ...  ...  ...   
Wycombe 2021   E1  2021-03-20   8.0  1.0  1.0  10.0  17.0  1.0  2.0  0.0  2.0   
                   2021-04-05  11.0  7.0  5.0  14.0  17.0  6.0  2.0  1.0  2.0   
                   2021-04-17  15.0  8.0  5.0   9.0  14.0  4.0  5.0  1.0  3.0   
                   2021-04-24  15.0  4.0  2.0  15.0  15.0  7.0  6.0  3.0  1.0   
                   2021-05-08  16.0  0.0  5.0  11.0  15.0  6.0  6.0  0.0  1.0   

                                HR   AR       HomeTeam  
Team    Season Div Date                                 
Ajaccio 2223   F1  2022-08-14  0.0  0.0            NaN  
                   2022-08-26  0.0  0.0            NaN  
                   2022-09-04  0.0  0.0            NaN  
                   2022-09-11  0.0  0.0            NaN  
                   2022-10-02  0.0  0.0            NaN  
...                            ...  ...            ...  
Wycombe 2021   E1  2021-03-20  0.0  0.0       Coventry  
                   2021-04-05  0.0  0.0      Rotherham  
                   2021-04-17  0.0  0.0        Swansea  
                   2021-04-24  1.0  0.0        Cardiff  
                   2021-05-08  0.0  0.0  Middlesbrough  

[26174 rows x 20 columns]

In [231]:
#We use the convenient combine_first method to combine both columns into a singular column, "Opposition", removing the NaN values.
df["Opposition"] = oppositionsDf["HomeTeam"].combine_first(oppositionsDf["AwayTeam"])
df

FTAG      HTAG         AS       AST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05  0.000000  0.000000   0.000000  0.000000   
                   2022-08-14       NaN       NaN        NaN       NaN   
                   2022-08-21  1.000000  1.000000   8.000000  4.000000   
                   2022-08-26       NaN       NaN        NaN       NaN   
                   2022-08-31  1.000000  0.500000   9.000000  3.500000   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17  0.800000  0.300000  10.150000  2.700000   
                   2021-04-21       NaN       NaN        NaN       NaN   
                   2021-04-24  0.857143  0.285714  10.380952  2.809524   
                   2021-05-01       NaN       NaN        NaN       NaN   
                   2021-05-08  0.863636  0.318182  10.590909  2.772727   

                                      AF        AC       AY        AR  \
Team    Season Div Date                                                 
Ajaccio 2223   F1  2022-08-05   0.000000  0.000000  0.00000  0.000000   
                   2022-08-14        NaN       NaN      NaN       NaN   
                   2022-08-21  18.000000  3.000000  6.00000  1.000000   
                   2022-08-26        NaN       NaN      NaN       NaN   
                   2022-08-31  14.500000  5.500000  4.00000  0.500000   
...                                  ...       ...      ...       ...   
Wycombe 2021   E1  2021-04-17  13.000000  3.700000  1.45000  0.100000   
                   2021-04-21        NaN       NaN      NaN       NaN   
                   2021-04-24  13.047619  3.761905  1.52381  0.095238   
                   2021-05-01        NaN       NaN      NaN       NaN   
                   2021-05-08  13.136364  3.863636  1.50000  0.090909   

                                   FTHG      HTHG         HS       HST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05       NaN       NaN        NaN       NaN   
                   2022-08-14  0.000000  0.000000   0.000000  0.000000   
                   2022-08-21       NaN       NaN        NaN       NaN   
                   2022-08-26  0.000000  0.000000   9.000000  2.000000   
                   2022-08-31       NaN       NaN        NaN       NaN   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17       NaN       NaN        NaN       NaN   
                   2021-04-21  0.666667  0.285714  10.238095  3.190476   
                   2021-04-24       NaN       NaN        NaN       NaN   
                   2021-05-01  0.727273  0.272727  10.772727  3.318182   
                   2021-05-08       NaN       NaN        NaN       NaN   

                                      HF        HC        HY        HR   Home  \
Team    Season Div Date                                                         
Ajaccio 2223   F1  2022-08-05        NaN       NaN       NaN       NaN  False   
                   2022-08-14   0.000000  0.000000  0.000000  0.000000   True   
                   2022-08-21        NaN       NaN       NaN       NaN  False   
                   2022-08-26  16.000000  1.000000  5.000000  0.000000   True   
                   2022-08-31        NaN       NaN       NaN       NaN  False   
...                                  ...       ...       ...       ...    ...   
Wycombe 2021   E1  2021-04-17        NaN       NaN       NaN       NaN  False   
                   2021-04-21  14.142857  4.666667  1.571429  0.095238   True   
                   2021-04-24        NaN       NaN       NaN       NaN  False   
                   2021-05-01  14.045455  4.681818  1.545455  0.090909   True   
                   2021-05-08        NaN       NaN       NaN       NaN  False   

                                  Opposition  
Team    Season Div Date                       
Ajaccio 222

In [255]:
"""import numpy as np         #We import Numpy for its select function, which allows us to easily set the values of a column based on some conditions; in this case, the values of the FTR and Home columns. 
conditions = [(df["Home"] == True) & (oppositionsDf["FTR"] == "H"), (df["Home"] == False) & (oppositionsDf["FTR"] == "A"), (oppositionsDf["FTR"] == "D")]
df["FTR"] = np.select(condlist=conditions, choicelist=["W", "W", "D"], default="L")"""
import numpy as np  
df["FTR"] = oppositionsDf["FTR"]
df

FTAG      HTAG         AS       AST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05  0.000000  0.000000   0.000000  0.000000   
                   2022-08-14       NaN       NaN        NaN       NaN   
                   2022-08-21  1.000000  1.000000   8.000000  4.000000   
                   2022-08-26       NaN       NaN        NaN       NaN   
                   2022-08-31  1.000000  0.500000   9.000000  3.500000   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17  0.800000  0.300000  10.150000  2.700000   
                   2021-04-21       NaN       NaN        NaN       NaN   
                   2021-04-24  0.857143  0.285714  10.380952  2.809524   
                   2021-05-01       NaN       NaN        NaN       NaN   
                   2021-05-08  0.863636  0.318182  10.590909  2.772727   

                                      AF        AC       AY        AR  \
Team    Season Div Date                                                 
Ajaccio 2223   F1  2022-08-05   0.000000  0.000000  0.00000  0.000000   
                   2022-08-14        NaN       NaN      NaN       NaN   
                   2022-08-21  18.000000  3.000000  6.00000  1.000000   
                   2022-08-26        NaN       NaN      NaN       NaN   
                   2022-08-31  14.500000  5.500000  4.00000  0.500000   
...                                  ...       ...      ...       ...   
Wycombe 2021   E1  2021-04-17  13.000000  3.700000  1.45000  0.100000   
                   2021-04-21        NaN       NaN      NaN       NaN   
                   2021-04-24  13.047619  3.761905  1.52381  0.095238   
                   2021-05-01        NaN       NaN      NaN       NaN   
                   2021-05-08  13.136364  3.863636  1.50000  0.090909   

                                   FTHG      HTHG         HS       HST  \
Team    Season Div Date                                                  
Ajaccio 2223   F1  2022-08-05       NaN       NaN        NaN       NaN   
                   2022-08-14  0.000000  0.000000   0.000000  0.000000   
                   2022-08-21       NaN       NaN        NaN       NaN   
                   2022-08-26  0.000000  0.000000   9.000000  2.000000   
                   2022-08-31       NaN       NaN        NaN       NaN   
...                                 ...       ...        ...       ...   
Wycombe 2021   E1  2021-04-17       NaN       NaN        NaN       NaN   
                   2021-04-21  0.666667  0.285714  10.238095  3.190476   
                   2021-04-24       NaN       NaN        NaN       NaN   
                   2021-05-01  0.727273  0.272727  10.772727  3.318182   
                   2021-05-08       NaN       NaN        NaN       NaN   

                                      HF        HC        HY        HR   Home  \
Team    Season Div Date                                                         
Ajaccio 2223   F1  2022-08-05        NaN       NaN       NaN       NaN  False   
                   2022-08-14   0.000000  0.000000  0.000000  0.000000   True   
                   2022-08-21        NaN       NaN       NaN       NaN  False   
                   2022-08-26  16.000000  1.000000  5.000000  0.000000   True   
                   2022-08-31        NaN       NaN       NaN       NaN  False   
...                                  ...       ...       ...       ...    ...   
Wycombe 2021   E1  2021-04-17        NaN       NaN       NaN       NaN  False   
                   2021-04-21  14.142857  4.666667  1.571429  0.095238   True   
                   2021-04-24        NaN       NaN       NaN       NaN  False   
                   2021-05-01  14.045455  4.681818  1.545455  0.090909   True   
                   2021-05-08        NaN       NaN       NaN       NaN  False   

                                  Opposition FTR  
Team    Season Div Date                           
Aja

In [239]:
#We again use the combine_first method, this time to fill the empty side(either Home or Away) with the stats by joining on an identically structured DataFrame indexed by the Opposition column instead of the Team index
teamsData = df.combine_first(df.reset_index().set_index(["Opposition", "Season", "Div", "Date"]).rename_axis(index=["Team", "Season", "Div", "Date"])).drop(columns=["Team"])
teamsData

AC         AF        AR         AS  \
Team    Season Div Date                                                   
Ajaccio 2223   F1  2022-08-05  0.000000   0.000000  0.000000   0.000000   
                   2022-08-14  5.000000  12.789474  0.210526  11.605263   
                   2022-08-21  3.000000  18.000000  1.000000   8.000000   
                   2022-08-26  4.444444  12.744444  0.177778  11.311111   
                   2022-08-31  5.500000  14.500000  0.500000   9.000000   
...                                 ...        ...       ...        ...   
Wycombe 2021   E1  2021-04-17  3.700000  13.000000  0.100000  10.150000   
                   2021-04-21  4.277778  11.977778  0.044444   9.977778   
                   2021-04-24  3.761905  13.047619  0.095238  10.380952   
                   2021-05-01  5.227848  10.151899  0.025316  11.253165   
                   2021-05-08  3.863636  13.136364  0.090909  10.590909   

                                    AST        AY      FTAG      FTHG FTR  \
Team    Season Div Date                                                     
Ajaccio 2223   F1  2022-08-05  0.000000  0.000000  0.000000  1.988764   H   
                   2022-08-14  3.842105  2.421053  1.421053  0.000000   D   
                   2022-08-21  4.000000  6.000000  1.000000  1.637363   H   
                   2022-08-26  4.288889  2.322222  1.277778  0.000000   A   
                   2022-08-31  3.500000  4.000000  1.000000  1.543478   H   
...                                 ...       ...       ...       ...  ..   
Wycombe 2021   E1  2021-04-17  2.700000  1.450000  0.800000  1.282353   D   
                   2021-04-21  3.388889  1.700000  1.255556  0.666667   H   
                   2021-04-24  2.809524  1.523810  0.857143  1.511628   H   
                   2021-05-01  4.025316  1.481013  1.215190  0.727273   H   
                   2021-05-08  2.772727  1.500000  0.863636  1.164835   A   

                                     HC         HF        HR         HS  \
Team    Season Div Date                                                   
Ajaccio 2223   F1  2022-08-05  6.179775  11.573034  0.067416  16.292135   
                   2022-08-14  0.000000   0.000000  0.000000   0.000000   
                   2022-08-21  6.120879  12.549451  0.054945  14.505495   
                   2022-08-26  1.000000  16.000000  0.000000   9.000000   
                   2022-08-31  5.369565  13.021739  0.119565  14.456522   
...                                 ...        ...       ...        ...   
Wycombe 2021   E1  2021-04-17  5.647059  10.164706  0.035294  12.905882   
                   2021-04-21  4.666667  14.142857  0.095238  10.238095   
                   2021-04-24  4.767442  12.813953  0.034884  13.197674   
                   2021-05-01  4.681818  14.045455  0.090909  10.772727   
                   2021-05-08  5.868132  12.131868  0.076923  13.714286   

                                    HST      HTAG      HTHG        HY   Home  \
Team    Season Div Date                                                        
Ajaccio 2223   F1  2022-08-05  6.337079  0.000000  0.988764  1.426966  False   
                   2022-08-14  0.000000  0.736842  0.000000  0.000000   True   
                   2022-08-21  4.989011  1.000000  0.703297  1.791209  False   
                   2022-08-26  2.000000  0.555556  0.000000  5.000000   True   
                   2022-08-31  5.076087  0.500000  0.663043  1.891304  False   
...                                 ...       ...       ...       ...    ...   
Wycombe 2021   E1  2021-04-17  4.388235  0.300000  0.494118  1.141176  False   
                   2021-04-21  3.190476  0.500000  0.285714  1.571429   True   
                   2021-04-24  4.546512  0.285714  0.674419  1.430233  False   
                   2021-05-01  3.318182  0.632911  0.272727  1.545455   True   
                   2021-05-08  3.901099  0.318182  0.637363  1.538462  False   

                                  Oppositi

Having resolved our DataFrame in the ways set out in the previous text cell, you may notice something is still off in our Table. The Seasons index level does not follow any pattern, and there are instances, such as the very first row, where we have a team with aggregated stats against a team with no stats. Both of these quirks can be easily explained:

- A team is not guaranteed to play in our requested divisions every single season. For instance, Ajaccio, as we can see from the first row of our DataFrame, has only played in Ligue 1 this season in the seasons since the 17/18 season. However, the seasons index level is still ordered on the Team level.
- We simply do not have data on a team that has not played in our considered divisions since the 17/18 season until that team's first match in the division in or after 17/18. This is problematic because any model will essentially have to treat these teams as essentially Random Variables, almost entirely removing its ability to predict their individual performance. A team's past performance in other divisions is also not very predictive of their performance in their current division; for instance, a team newly promoted to the English Premier League obviously performed very strongly in the Championship (well enough to be promoted) but will usually place in the bottom 10 by the end of the season. With this in mind, we will not gather additional data for these teams.

In the following code cells, we will perform some basic exploratory analysis on the data.

In [240]:
teamsData.corr().style.background_gradient()

C:\Users\jonat\AppData\Local\Temp\ipykernel_30336\1723862977.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  teamsData.corr().style.background_gradient()


,AC,AF,AR,AS,AST,AY,FTAG,FTHG,HC,HF,HR,HS,HST,HTAG,HTHG,HY,Home
AC,1.000000,0.153185,0.004627,0.755280,0.660536,0.065285,0.524709,0.117133,0.191378,0.158752,-0.005669,0.193931,0.176077,0.437899,0.104789,0.077669,-0.000000
AF,0.153185,1.000000,0.232431,0.163289,0.103830,0.673531,0.003851,0.132598,0.163258,0.447392,0.151352,0.199761,0.213991,-0.006712,0.116896,0.345739,0.000000
AR,0.004627,0.232431,1.000000,0.011189,0.043246,0.249975,0.012509,0.014571,0.022484,0.164027,0.139158,0.005857,0.064654,0.001042,0.029835,0.138118,0.000000
AS,0.755280,0.163289,0.011189,1.000000,0.816271,0.040982,0.685822,0.134926,0.185969,0.183919,-0.003395,0.250640,0.163299,0.561091,0.121854,0.044523,-0.000000
AST,0.660536,0.103830,0.043246,0.816271,1.000000,0.154632,0.809209,0.123359,0.140009,0.202263,0.050962,0.157234,0.207800,0.657631,0.109515,0.138746,-0.000000
AY,0.065285,0.673531,0.249975,0.040982,0.154632,1.000000,0.068988,0.078522,0.105925,0.417315,0.202037,0.079408,0.165710,0.035004,0.051414,0.463742,0.000000
FTAG,0.524709,0.003851,0.012509,0.685822,0.809209,0.068988,1.000000,0.092937,0.106188,0.128559,0.031131,0.131009,0.133304,0.838526,0.076462,0.085575,-0.000000
FTHG,0.117133,0.132598,0.014571,0.134926,0.123359,0.078522,0.092937,1.000000,0.535455,-0.037213,-0.009016,0.676049,0.819441,0.083414,0.865221,-0.099004,-0.000000
HC,0.191378,0.163258,0.022484,0.185969,0.140009,0.105925,0.106188,0.535455,1.000000,0.120855,0.003903,0.779727,0.672062,0.105038,0.436420,0.005358,-0.000000
HF,0.158752,0.447392,0.164027,0.183919,0.202263,0.417315,0.128559,-0.037213,0.120855,1.000000,0.238944,0.146312,0.094063,0.089348,-0.048192,0.697940,0.000000


In [241]:
matchDf.sort_index(axis=1).corr().style.background_gradient()

C:\Users\jonat\AppData\Local\Temp\ipykernel_30336\1495832878.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  matchDf.sort_index(axis=1).corr().style.background_gradient()


,AC,AF,AR,AS,AST,AY,FTAG,FTHG,HC,HF,HR,HS,HST,HTAG,HTHG,HY
AC,1.000000,-0.067209,-0.074224,0.488163,0.289886,-0.074204,0.035017,-0.051034,-0.243139,-0.023770,0.056847,-0.279014,-0.155341,-0.035488,0.038698,0.029616
AF,-0.067209,1.000000,0.067110,-0.090583,-0.050559,0.364850,-0.028679,-0.062754,-0.049803,0.145823,0.050150,-0.060231,-0.037903,-0.002203,-0.060089,0.107767
AR,-0.074224,0.067110,1.000000,-0.110058,-0.084437,0.163273,-0.069003,0.064494,0.059459,0.039186,0.082036,0.110253,0.094618,-0.024078,0.021470,0.072268
AS,0.488163,-0.090583,-0.110058,1.000000,0.654330,-0.096124,0.310780,-0.104016,-0.261861,-0.019438,0.118032,-0.292367,-0.207687,0.153980,0.000822,0.054294
AST,0.289886,-0.050559,-0.084437,0.654330,1.000000,-0.051580,0.583855,-0.098566,-0.147200,-0.006357,0.116114,-0.198178,-0.126899,0.377341,-0.032760,0.065136
AY,-0.074204,0.364850,0.163273,-0.096124,-0.051580,1.000000,-0.018797,-0.009514,0.022187,0.114500,0.079242,0.030364,0.032515,-0.004902,-0.015990,0.220685
FTAG,0.035017,-0.028679,-0.069003,0.310780,0.583855,-0.018797,1.000000,-0.083672,-0.045175,-0.017890,0.118376,-0.086480,-0.097535,0.687165,-0.044701,0.060051
FTHG,-0.051034,-0.062754,0.064494,-0.104016,-0.098566,-0.009514,-0.083672,1.000000,0.015610,-0.092251,-0.078455,0.274683,0.577721,-0.038871,0.687128,-0.108616
HC,-0.243139,-0.049803,0.059459,-0.261861,-0.147200,0.022187,-0.045175,0.015610,1.000000,-0.101664,-0.060576,0.498646,0.285140,0.038273,-0.062958,-0.059881
HF,-0.023770,0.145823,0.039186,-0.019438,-0.006357,0.114500,-0.017890,-0.092251,-0.101664,1.000000,0.049182,-0.132308,-0.084599,-0.013374,-0.044676,0.379417


In [263]:
teamsData[teamsData.isna().any(axis=1)]

,,,,AC,AF,AR,AS,AST,AY,FTAG,FTHG,FTR,HC,HF,HR,HS,HST,HTAG,HTHG,HY,Home,Opposition
Team,Season,Div,Date,,,,,,,,,,,,,,,,,,,


Some insights from the above DataFrames:
## teamsData.describe()
- The mean goal difference between the Home and Away sides at half time (HTHG - HTAG) is ~0.13 goals in favour of the home side.
- This average lead is more than doubled to almost exactly 0.3 goals, again in favour of the home side, at full time (FTHG - FTAG).
- The Home team, on average, gets just under 1 (0.96) more corner than the Away team (HC mean - AC mean).

## teamsData.corr()
The DataFrame corr method, by default, computes the PMCC of every column in relation to every other column. It is important to remember that numeric columns in the teamsData DataFrame are season-wide cumulative means and therefore the correlations are between those and not say, the correlations between any 2 statistics in any particular match.
- Interestingly, Home and Away fouls in our dataset have a PMCC of 0.45 to 2 d.p - seems footballers do fight back.
- Yellow cards, Red cards and fouls in general are positively correlated (with magnitude ~< 0.1) with that same team's shots, shots on target and even corners! On the other hand, they have virtually ne correlation with a team's goals. My interpretation of this is that players, and therefore teams (as a collective of players), tend at least marginally to become frustrated or desperate as shots and shots on target are not converted to goals. 


In [267]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

idx = pd.IndexSlice
clf = LogisticRegression()

def toTrainTest(df: pd.DataFrame, lastTrainSeason: int = 2122, mapLabelsToInt: bool = False):
    trainingData = [df.drop(columns = ["FTR", "Opposition"]).loc[idx[:, :lastTrainSeason, :, :]], df.loc[idx[:, :lastTrainSeason, :, :], "FTR"]]
    testData = [df.drop(columns = ["FTR", "Opposition"]).loc[idx[:, lastTrainSeason + 101:, :, :]], df.loc[idx[:, lastTrainSeason + 101:, :, :], "FTR"]]

    trainingData[0]["Home"] = df["Home"].loc[idx[:, :lastTrainSeason, :, :]].copy()
    #trainingData[0]["Avg Points Vs Opposition"] = df.loc[idx[:, :lastTrainSeason, :, :]].sort_values(by=["Team", "Opposition"])["FTR"].map({"H": 3, "D": 1, "A":0}).expanding().mean().copy()
    testData[0]["Home"] = df["Home"].loc[idx[:, lastTrainSeason + 101:, :, :]].copy()
    #testData[0]["Avg Points Vs Opposition"] = df.loc[idx[:, lastTrainSeason + 101:, :, :]].sort_values(by=["Team", "Opposition"])["FTR"].map({"H": 3, "D": 1, "A":0}).expanding().mean().copy()
    if mapLabelsToInt:
        trainingData[1] = trainingData[1].map({"H": 2, "D": 1, "A":0})
        testData[1] = testData[1].map({"H": 2, "D": 1, "A":0})
    return trainingData, testData

trainingData, testData = toTrainTest(df = teamsData, mapLabelsToInt=True)
clf = clf.fit(*trainingData)
clf.score(*testData)

precision_recall_fscore_support(y_true = testData[1], y_pred = clf.predict(testData[0]), labels = )

d:\PythonInstallation\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(array([0.47543353, 0.        , 0.48633018]),
 array([0.39448441, 0.        , 0.82885305]),
 array([0.43119266, 0.        , 0.61298873]),
 array([ 834,  646, 1116], dtype=int64))